In [1]:
pip install scikit-learn pandas numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\shahb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import pickle

# Load dataset
movies_df = pd.read_csv('tmdb_5000_movies.csv')

# Handling missing values in 'runtime'
movies_df['runtime'] = movies_df['runtime'].fillna(movies_df['runtime'].mean())

# Selecting features and target
X = movies_df[['budget', 'runtime']]
y = movies_df['popularity']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = RandomForestRegressor(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# Save the model to disk
filename = 'movie_popularity_model.pkl'
pickle.dump(model, open(filename, 'wb'))

# Load the trained model
model = pickle.load(open(filename, 'rb'))

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Print R-squared
print("R-squared:", r2)


R-squared: 0.14201504740032989


In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pickle
import numpy as np  # Import NumPy here

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for the entire app

# Load the trained model
filename = 'movie_popularity_model.pkl'
model = pickle.load(open(filename, 'rb'))

@app.route('/predict', methods=['POST'])
def predict_popularity():
    data = request.get_json(force=True)
    budget = data['budget']
    runtime = data['runtime']
    
    # Make prediction
    prediction = model.predict(np.array([[budget, runtime]]))  # Use NumPy here
    
    # Return the prediction
    return jsonify(popularity=prediction[0])

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
